In [ ]:
DATA PREPROCESSING

In [5]:

#Importing modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder

In [19]:
#Importing data and defining preprocessing function
train = pd.read_csv('labeledTrainData.tsv', header=0, delimiter='\t', quoting=3 )
def review_to_words(raw_review):
    bs = BeautifulSoup(raw_review).get_text()
    letters_only = re.sub('[^a-zA-Z]', ' ', bs)
    words_only = letters_only.lower().split() #tokenization
    stops = set(stopwords.words('english'))
    words = [w for w in words_only if not w in stops]
    return (" ".join(words))

In [38]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [138]:
train['review'][1]

'"\\"The Classic War of the Worlds\\" by Timothy Hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate H. G. Wells\' classic book. Mr. Hines succeeds in doing so. I, and those who watched his film with me, appreciated the fact that it was not the standard, predictable Hollywood fare that comes out every year, e.g. the Spielberg version with Tom Cruise that had only the slightest resemblance to the book. Obviously, everyone looks for different things in a movie. Those who envision themselves as amateur \\"critics\\" look only to criticize everything they can. Others rate a movie on more important bases,like being entertained, which is why most people never agree with the \\"critics\\". We enjoyed the effort Mr. Hines put into being faithful to H.G. Wells\' classic novel, and we found it to be very entertaining. This made it easy to overlook what the \\"critics\\" perceive to be its shortcomings."'

In [20]:
num_review = train['review'].size #5000
clean_train_review = []
#cleaning the reviews
for i in xrange(0, num_review):
    clean_train_review.append(review_to_words(train['review'][i]))

In [139]:
clean_train_review[50]

u'stephane rideau already star tour de force wild reeds one france biggest indie stars film plays cedric local boy meets vacationing mathieu newcomer jamie elkaim stunning nuanced ethereal performance beach mathieu complex relationship ill mother demanding aunt sister competitive relationship soon two falling love film fractured narrative comprised lengthy flash backs bits pieces present real time forward movement future little daunting director sebastien lifshitz signal time period story line difficult follow stick film final minutes engrossing able take eyes screen turns heart breaking uplifting film ranks beautiful thing must see cinema'

In [21]:
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 
train_data_features = vectorizer.fit_transform(clean_train_review)
#convert result to array, cos of their ease to work with
train_data_features = train_data_features.toarray()
print (train_data_features.shape)

(25000, 5000)


In [36]:
train_data_features.size

125000000

In [42]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_data_features, train.sentiment, test_size = 0.2)


In [65]:
x_train.shape

(20000, 5000)

In [44]:
x_test.shape

(5000, 5000)

In [86]:
y_train.head()

10953    0
18804    1
8213     1
16587    0
6171     0
Name: sentiment, dtype: int64

In [79]:
#Importing tensorflow
import tensorflow as tf
#defining hyperparameters
learning_rate = 0.01
training_epochs = 1000
batch_size = 100
num_labels = 2
num_features = 5000
train_size = 20000

In [80]:
#defining the placeholders and encoding the y placeholder
X = tf.placeholder(tf.float32, shape=[None, num_features])
Y = tf.placeholder(tf.int32, shape=[None])
y_oneHot = tf.one_hot(Y, 1)

#defining the model parameters -- weight and bias
W = tf.Variable(tf.zeros([num_features, 1]))
b = tf.Variable(tf.zeros([1]))

#defining the sigmoid model and setting up the learning algorithm
y_model = tf.nn.sigmoid(tf.add(tf.matmul(X, W), b))
cost = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_model, labels=y_oneHot)
train_optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

#defining operation to measure success rate
correct_prediction = tf.equal(tf.argmax(y_model, 1), tf.argmax(y_oneHot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#saving variables
saver = tf.train.Saver()

In [69]:
import os
os.getcwd()

'/Users/oluwolealowolodu/Documents/Post botcamp practise'

In [81]:
#executing the graph and saving the model variables
with tf.Session() as sess: #opening a new session
    tf.global_variables_initializer().run()
    
    #Iteratively updating parameter batch by batch
    for step in range(training_epochs * train_size // batch_size):
        offset = (step * batch_size) % train_size
        batch_xs = x_train[offset:(offset + batch_size), :]
        batch_labels = y_train[offset:(offset + batch_size)]
        #run optimizer on batch
        err, _ = sess.run([cost, train_optimizer], feed_dict={X:batch_xs, Y:batch_labels})
        if step % 1000 ==0:
            print(step, err) #print ongoing result
    #Print final learned parameters
    w_val = sess.run(W)
    print('w', w_val)
    b_val = sess.run(b)
    print('b', b_val)
    print('Accuracy', accuracy.eval(feed_dict={X:x_test, Y:y_test}))
    save_path = saver.save(sess, './movie_review_variables')
    print('Model saved in path {}'.format(save_path))
    


(0, array([[0.474077],
       [0.974077],
       [0.974077],
       [0.474077],
       [0.474077],
       [0.974077],
       [0.474077],
       [0.974077],
       [0.474077],
       [0.974077],
       [0.474077],
       [0.474077],
       [0.974077],
       [0.974077],
       [0.474077],
       [0.974077],
       [0.974077],
       [0.474077],
       [0.974077],
       [0.474077],
       [0.474077],
       [0.474077],
       [0.474077],
       [0.974077],
       [0.474077],
       [0.474077],
       [0.474077],
       [0.974077],
       [0.974077],
       [0.974077],
       [0.474077],
       [0.974077],
       [0.974077],
       [0.474077],
       [0.474077],
       [0.474077],
       [0.474077],
       [0.474077],
       [0.974077],
       [0.974077],
       [0.474077],
       [0.474077],
       [0.974077],
       [0.974077],
       [0.974077],
       [0.974077],
       [0.474077],
       [0.974077],
       [0.474077],
       [0.974077],
       [0.974077],
       [0.474077],
       [

(4000, array([[0.50775   ],
       [0.76291084],
       [0.69517815],
       [0.3132617 ],
       [0.3132617 ],
       [0.8530168 ],
       [0.40748245],
       [0.69314766],
       [0.31333473],
       [0.7244395 ],
       [0.31516716],
       [0.35395202],
       [0.7096588 ],
       [1.313002  ],
       [0.32388622],
       [0.7774942 ],
       [0.69446135],
       [0.3210605 ],
       [0.70101184],
       [0.35068002],
       [0.31326497],
       [0.66501594],
       [0.3310118 ],
       [0.76976347],
       [0.6806117 ],
       [0.31343588],
       [0.3400852 ],
       [0.69864464],
       [0.6931484 ],
       [0.71498847],
       [0.31326532],
       [0.8328451 ],
       [0.6932589 ],
       [0.69004893],
       [0.3132617 ],
       [0.33486038],
       [0.31327116],
       [0.42966887],
       [0.6974807 ],
       [0.69897056],
       [0.31326735],
       [0.31364572],
       [1.0533949 ],
       [0.7272898 ],
       [0.7029525 ],
       [1.029783  ],
       [0.39942622],
      

(8000, array([[0.46710935],
       [0.76131344],
       [0.693593  ],
       [0.3132617 ],
       [0.3132617 ],
       [0.78054965],
       [0.36688125],
       [0.6931472 ],
       [0.3132957 ],
       [0.7242316 ],
       [0.3137853 ],
       [0.34203956],
       [0.7027561 ],
       [1.3131558 ],
       [0.31932685],
       [0.7479571 ],
       [0.69403726],
       [0.31895578],
       [0.6966729 ],
       [0.34735498],
       [0.31326184],
       [0.66957736],
       [0.32054603],
       [0.7463758 ],
       [0.68225485],
       [0.31329656],
       [0.32790282],
       [0.70307565],
       [0.69314754],
       [0.70202965],
       [0.31326225],
       [0.8308506 ],
       [0.6931618 ],
       [0.6863405 ],
       [0.3132617 ],
       [0.32457158],
       [0.31326273],
       [0.40565577],
       [0.69576055],
       [0.69567096],
       [0.31326216],
       [0.3133328 ],
       [0.88803834],
       [0.7224314 ],
       [0.6980088 ],
       [1.0359665 ],
       [0.37540397],
      

(12000, array([[0.43727675],
       [0.7494042 ],
       [0.69329274],
       [0.3132617 ],
       [0.3132617 ],
       [0.7505895 ],
       [0.3534779 ],
       [0.6931472 ],
       [0.31330433],
       [0.72450584],
       [0.31343567],
       [0.34059447],
       [0.7001901 ],
       [1.3132038 ],
       [0.31753936],
       [0.7307054 ],
       [0.6939448 ],
       [0.31806418],
       [0.6950103 ],
       [0.34295014],
       [0.31326172],
       [0.6762736 ],
       [0.31858638],
       [0.73605824],
       [0.6850922 ],
       [0.31327575],
       [0.32129437],
       [0.70634437],
       [0.6931473 ],
       [0.69747674],
       [0.31326184],
       [0.81627446],
       [0.69314873],
       [0.6824725 ],
       [0.3132617 ],
       [0.3217525 ],
       [0.31326187],
       [0.38235715],
       [0.6954586 ],
       [0.6942158 ],
       [0.31326175],
       [0.31328735],
       [0.80108166],
       [0.7192621 ],
       [0.6968584 ],
       [0.9936981 ],
       [0.35399488],
     

(16000, array([[0.42073238],
       [0.7370883 ],
       [0.69320965],
       [0.3132617 ],
       [0.3132617 ],
       [0.73412085],
       [0.34342805],
       [0.6931472 ],
       [0.31332815],
       [0.7209461 ],
       [0.3133371 ],
       [0.34307438],
       [0.69858414],
       [1.3132223 ],
       [0.31654412],
       [0.72288406],
       [0.6938128 ],
       [0.31723967],
       [0.69437367],
       [0.3314995 ],
       [0.3132617 ],
       [0.6810634 ],
       [0.317419  ],
       [0.73002625],
       [0.68679446],
       [0.3132692 ],
       [0.31909767],
       [0.7083087 ],
       [0.6931473 ],
       [0.6957921 ],
       [0.31326172],
       [0.7986479 ],
       [0.69314754],
       [0.6786025 ],
       [0.3132617 ],
       [0.31902292],
       [0.31326175],
       [0.36761478],
       [0.6953453 ],
       [0.6936947 ],
       [0.31326172],
       [0.3132728 ],
       [0.77001333],
       [0.71709883],
       [0.69601965],
       [0.9458511 ],
       [0.34110954],
     

...

(184000, array([[0.32580614],
       [0.69610155],
       [0.69314724],
       [0.3132617 ],
       [0.3132617 ],
       [0.7033968 ],
       [0.32897535],
       [0.6931472 ],
       [0.31329998],
       [0.7000772 ],
       [0.3132617 ],
       [0.31829414],
       [0.693362  ],
       [1.313261  ],
       [0.31343606],
       [0.697559  ],
       [0.6931551 ],
       [0.31333655],
       [0.6931473 ],
       [0.31327343],
       [0.3132617 ],
       [0.6916611 ],
       [0.31340882],
       [0.70100236],
       [0.6929859 ],
       [0.31326172],
       [0.31329277],
       [0.7006288 ],
       [0.6931472 ],
       [0.6931505 ],
       [0.3132617 ],
       [0.7061486 ],
       [0.6931472 ],
       [0.31400037],
       [0.3132617 ],
       [0.3133467 ],
       [0.3132617 ],
       [0.3166198 ],
       [0.6944469 ],
       [0.6931479 ],
       [0.3132617 ],
       [0.3132617 ],
       [0.7062391 ],
       [0.7107185 ],
       [0.69315165],
       [0.7248168 ],
       [0.31846645],
    

(188000, array([[0.32560953],
       [0.6960438 ],
       [0.69314724],
       [0.3132617 ],
       [0.3132617 ],
       [0.7033906 ],
       [0.32834134],
       [0.6931472 ],
       [0.31330296],
       [0.7000134 ],
       [0.3132617 ],
       [0.31841442],
       [0.69333905],
       [1.313261  ],
       [0.31342655],
       [0.6974809 ],
       [0.69315475],
       [0.3133321 ],
       [0.6931473 ],
       [0.3132726 ],
       [0.3132617 ],
       [0.69165593],
       [0.31340283],
       [0.70063627],
       [0.6929931 ],
       [0.3132617 ],
       [0.3132908 ],
       [0.70053023],
       [0.6931472 ],
       [0.69315016],
       [0.3132617 ],
       [0.7058609 ],
       [0.6931472 ],
       [0.31395972],
       [0.3132617 ],
       [0.31334448],
       [0.3132617 ],
       [0.31664357],
       [0.6943817 ],
       [0.6931479 ],
       [0.3132617 ],
       [0.3132617 ],
       [0.705873  ],
       [0.71083117],
       [0.69315153],
       [0.72428346],
       [0.31837645],
    

(192000, array([[0.32545435],
       [0.6959728 ],
       [0.6931472 ],
       [0.3132617 ],
       [0.3132617 ],
       [0.7032703 ],
       [0.32736808],
       [0.6931472 ],
       [0.31330553],
       [0.69992995],
       [0.3132617 ],
       [0.31851777],
       [0.69331956],
       [1.3132612 ],
       [0.31341898],
       [0.6974    ],
       [0.6931546 ],
       [0.3133273 ],
       [0.6931473 ],
       [0.3132718 ],
       [0.3132617 ],
       [0.6916464 ],
       [0.31339762],
       [0.7002368 ],
       [0.69299805],
       [0.31326172],
       [0.31328878],
       [0.7004376 ],
       [0.6931472 ],
       [0.6931498 ],
       [0.3132617 ],
       [0.705465  ],
       [0.6931472 ],
       [0.31390673],
       [0.3132617 ],
       [0.31334248],
       [0.3132617 ],
       [0.31665576],
       [0.6943164 ],
       [0.69314784],
       [0.3132617 ],
       [0.3132617 ],
       [0.7054354 ],
       [0.710878  ],
       [0.69315153],
       [0.7236907 ],
       [0.3181843 ],
    

(196000, array([[0.32541016],
       [0.69589996],
       [0.6931473 ],
       [0.3132617 ],
       [0.3132617 ],
       [0.7029932 ],
       [0.32589483],
       [0.6931472 ],
       [0.31330818],
       [0.6997676 ],
       [0.3132617 ],
       [0.31857952],
       [0.693303  ],
       [1.313261  ],
       [0.31341535],
       [0.6973664 ],
       [0.6931547 ],
       [0.3133219 ],
       [0.6931473 ],
       [0.313271  ],
       [0.3132617 ],
       [0.6916303 ],
       [0.31339392],
       [0.69978464],
       [0.6929997 ],
       [0.31326172],
       [0.31328672],
       [0.70037884],
       [0.6931472 ],
       [0.69314945],
       [0.3132617 ],
       [0.70481926],
       [0.6931472 ],
       [0.3138289 ],
       [0.3132617 ],
       [0.31334063],
       [0.3132617 ],
       [0.31670812],
       [0.6942463 ],
       [0.69314784],
       [0.3132617 ],
       [0.3132617 ],
       [0.704849  ],
       [0.71080494],
       [0.69315165],
       [0.72302866],
       [0.3177901 ],
    

('w', array([[ 0.13328114],
       [-0.8262448 ],
       [-0.96044886],
       ...,
       [ 0.90239793],
       [ 1.8408026 ],
       [ 0.44223914]], dtype=float32))
('b', array([-0.49304736], dtype=float32))
('Accuracy', 1.0)
Model saved in path ./movie_review_variables


IMPORTING THE UNLABELED TEST FILE

In [ ]:
#reading test file 
test = pd.read_csv('testData.tsv', delimiter='\t', header=0, quoting=3)

In [35]:
test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [82]:
#reading test file 
test = pd.read_csv('testData.tsv', delimiter='\t', header=0, quoting=3)
#appending the review to a list
num2_reviews = len(test['review'])
clean_test_review = []
for i in xrange(0, num2_reviews):
    clean_test_review.append(review_to_words(test['review'][i]))

#converting the bag of words of test to numpy array
test_data_features = vectorizer.transform(clean_test_review)
test_data_features = test_data_features.toarray()
print test_data_features.shape


(25000, 5000)


In [124]:
#creating csv file for kaggle submission
with tf.Session() as sess:
    saver.restore(sess, './movie_review_variables')
    predictions = sess.run(y_model, feed_dict={X: test_data_features})
    subm2 = pd.DataFrame(data={'id':test['id'], 'sentiment':predictions[:,0]})
    subm2.to_csv('subm2nlp.csv', index=False, quoting=3)
    print("I am done predicting")
    # '''Public Score 0.84760 in position 345 on first submision'''

INFO:tensorflow:Restoring parameters from ./movie_review_variables
I am done predicting


In [125]:
import pandas as pd
sub2 = pd.read_csv('subm2nlp.csv')
sub2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
id           25000 non-null object
sentiment    25000 non-null float64
dtypes: float64(1), object(1)
memory usage: 390.7+ KB


In [99]:
sub2.head(20)

,id,sentiment
0,12311_10,3.874302e-07
1,8348_2,9.999982e-01
2,5828_4,1.765098e-01
3,7186_2,2.095658e-02
4,12128_7,0.000000e+00
5,2913_8,9.455299e-01
6,4396_1,9.999983e-01
7,395_2,8.515527e-01
8,10616_1,1.000000e+00
9,9074_9,9.999998e-01


In [131]:
#setting treshold for predictions and exporting to csv file
q = 0.5
sub2.sentiment = q * np.round(sub2['sentiment']/q)
sub2.sentiment.replace({0.5:1,0.0:0,1.0:1}, inplace=True)
sub2.sentiment = sub2.sentiment.astype(int)
sub2.to_csv('subm2nlp.csv', index=False, quoting=3)


In [ ]:
INFO:tensorflow:Restoring parameters from ./movie_review_variables
Prections are [3.8743019e-07 9.9999821e-01 1.7650980e-01 ... 9.9997473e-01 1.4901161e-07
 7.0333481e-06]